# Validation using various techniques

- K fold cross validation

It consists of splitting the available data into K partitions (typically = 4 or 5), instantiating K
identical models, and training each one on (K – 1) partitions while evaluating on the remaining partition.
The validation score for the model used is then the average of the K validation scores obtained.

<img src="../images/Screenshot 2021-12-16 123443.jpg">

In [54]:
import numpy as np
import pandas as pd
import seaborn as sns

from matplotlib import pyplot as plt
from sklearn.model_selection import KFold

from tensorflow.keras.datasets import boston_housing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense
from tensorflow.keras.activations import relu
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import mean_squared_error
from tensorflow.keras.metrics import mean_absolute_error
from tensorflow.keras.utils import plot_model

### create model

In [55]:
def create_model(shape):
    # layers of the neural net
    input_layer = Dense(units=64, activation=relu, input_shape=shape)
    hidden_1 = Dense(units=64, activation=relu)
    output_layer = Dense(units=1)

    # compilation
    model = Sequential([input_layer, hidden_1, output_layer])
    model.compile(optimizer=RMSprop(learning_rate=0.01),
                  loss=mean_squared_error,
                  metrics=[mean_absolute_error])

    return model

### training

In [42]:
(X, Y), (test_x, test_y) = boston_housing.load_data(path='../data')
X.shape, Y.shape, test_x.shape, test_y.shape

((404, 13), (404,), (102, 13), (102,))

In [13]:
create_model(shape=(13,)).summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                896       
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5,121
Trainable params: 5,121
Non-trainable params: 0
_________________________________________________________________


In [20]:
plot_model(create_model(shape=(13,)))

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


### cross validation

In [56]:
scores = []
cv = KFold(n_splits=4, shuffle=True, random_state=64)

for fold, (train_id, val_id) in enumerate(cv.split(X, Y)):
    train_x, val_x = X[train_id], X[val_id]
    train_y, val_y = Y[train_id], Y[val_id]

    network = create_model(shape=(train_x.shape[1],))
    network.fit(train_x, train_y,
                epochs=10,
                batch_size=1,
                validation_data=(val_x, val_y))

    predicted_y = network.predict(val_x)
    scores.append(mean_absolute_error(val_y, predicted_y))

Epoch 1/10
303/303 [==============================] - 2s 4ms/step - loss: 909.8812 - mean_absolute_error: 12.9236 - val_loss: 71.2876 - val_mean_absolute_error: 6.0755
Epoch 2/10
303/303 [==============================] - 1s 4ms/step - loss: 87.0515 - mean_absolute_error: 6.5198 - val_loss: 89.5325 - val_mean_absolute_error: 6.3189
Epoch 3/10
303/303 [==============================] - 1s 4ms/step - loss: 72.4509 - mean_absolute_error: 6.0255 - val_loss: 524.3302 - val_mean_absolute_error: 21.4998
Epoch 4/10
303/303 [==============================] - 1s 4ms/step - loss: 60.4064 - mean_absolute_error: 5.5592 - val_loss: 50.6981 - val_mean_absolute_error: 4.6020
Epoch 5/10
303/303 [==============================] - 2s 6ms/step - loss: 58.8880 - mean_absolute_error: 5.3898 - val_loss: 72.8595 - val_mean_absolute_error: 5.5098
Epoch 6/10
303/303 [==============================] - 2s 7ms/step - loss: 54.9295 - mean_absolute_error: 5.3101 - val_loss: 56.3913 - val_mean_absolute_error: 5.0952
